# Human-in-the-Loop Workflow med Microsoft Agent Framework

## 🎯 Læringsmål

I denne notebook lærer du, hvordan du implementerer **human-in-the-loop** workflows ved hjælp af Microsoft Agent Framework's `RequestInfoExecutor`. Dette kraftfulde mønster giver dig mulighed for at pause AI-workflows for at indhente menneskelig input, hvilket gør dine agenter interaktive og giver mennesker kontrol over kritiske beslutninger.

## 🔄 Hvad er Human-in-the-Loop?

**Human-in-the-loop (HITL)** er et designmønster, hvor AI-agenter pauser udførelsen for at anmode om menneskelig input, før de fortsætter. Dette er essentielt for:

- ✅ **Kritiske beslutninger** - Få menneskelig godkendelse, før der træffes vigtige handlinger
- ✅ **Tvetydige situationer** - Lad mennesker afklare, når AI er usikker
- ✅ **Brugerpræferencer** - Spørg brugere om at vælge mellem flere muligheder
- ✅ **Overholdelse & sikkerhed** - Sikr menneskelig overvågning af regulerede operationer
- ✅ **Interaktive oplevelser** - Byg samtaleagenter, der reagerer på brugerinput

## 🏗️ Sådan fungerer det i Microsoft Agent Framework

Frameworket tilbyder tre nøglekomponenter til HITL:

1. **`RequestInfoExecutor`** - En speciel executor, der pauser workflowet og udsender en `RequestInfoEvent`
2. **`RequestInfoMessage`** - Basisklasse for typede anmodningspayloads sendt til mennesker
3. **`RequestResponse`** - Knytter menneskelige svar til de oprindelige anmodninger ved hjælp af `request_id`

**Workflow-mønster:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Vores eksempel: Hotelbooking med brugerbekræftelse

Vi bygger videre på det betingede workflow ved at tilføje menneskelig bekræftelse **før** vi foreslår alternative destinationer:

1. Brugeren anmoder om en destination (f.eks. "Paris")
2. `availability_agent` tjekker, om der er ledige værelser
3. **Hvis ingen værelser** → `confirmation_agent` spørger "Vil du se alternativer?"
4. Workflowet **pauser** ved hjælp af `RequestInfoExecutor`
5. **Mennesket svarer** "ja" eller "nej" via konsolinput
6. `decision_manager` styrer baseret på svaret:
   - **Ja** → Vis alternative destinationer
   - **Nej** → Annuller bookinganmodningen
7. Vis det endelige resultat

Dette demonstrerer, hvordan man giver brugerne kontrol over agentens forslag!

---

Lad os komme i gang! 🚀


## Trin 1: Importer nødvendige biblioteker

Vi importerer de standardkomponenter fra Agent Framework plus **specifikke klasser til menneskelig indblanding**:
- `RequestInfoExecutor` - Executor, der pauser arbejdsgangen for menneskelig input
- `RequestInfoEvent` - Begivenhed, der udløses, når menneskelig input anmodes
- `RequestInfoMessage` - Basisklasse for typede anmodningsdata
- `RequestResponse` - Knytter menneskelige svar til anmodninger
- `WorkflowOutputEvent` - Begivenhed til at registrere arbejdsgangens output


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Trin 2: Definer Pydantic-modeller for strukturerede outputs

Disse modeller definerer **skemaet**, som agenterne vil returnere. Vi beholder alle modeller fra den betingede arbejdsgang og tilføjer:

**Nyt for Human-in-the-Loop:**
- `HumanFeedbackRequest` - Underklasse af `RequestInfoMessage`, der definerer anmodningspayloaden, som sendes til mennesker
  - Indeholder `prompt` (spørgsmål, der skal stilles) og `destination` (kontekst om den utilgængelige by)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Trin 3: Opret værktøjet til hotelbooking

Samme værktøj som i den betingede arbejdsgang - tjekker om der er ledige værelser på destinationen.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Trin 4: Definer betingelsesfunktioner for routing

Vi har brug for **fire betingelsesfunktioner** til vores human-in-the-loop arbejdsproces:

**Fra betinget arbejdsproces:**
1. `has_availability_condition` - Ruter, når hoteller ER tilgængelige
2. `no_availability_condition` - Ruter, når hoteller IKKE er tilgængelige

**Nyt for human-in-the-loop:**
3. `user_wants_alternatives_condition` - Ruter, når brugeren siger "ja" til alternativer
4. `user_declines_alternatives_condition` - Ruter, når brugeren siger "nej" til alternativer


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Trin 5: Opret Decision Manager Executor

Dette er **kernen i human-in-the-loop mønsteret**! `DecisionManager` er en skræddersyet `Executor`, der:

1. **Modtager menneskelig feedback** via `RequestResponse` objekter
2. **Behandler brugerens beslutning** (ja/nej)
3. **Styrer arbejdsgangen** ved at sende beskeder til de relevante agenter

Nøglefunktioner:
- Bruger `@handler` dekoratoren til at eksponere metoder som arbejdsgangstrin
- Modtager `RequestResponse[HumanFeedbackRequest, str]`, der indeholder både den oprindelige anmodning og brugerens svar
- Genererer simple "ja" eller "nej" beskeder, der udløser vores betingelsesfunktioner


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Trin 6: Opret brugerdefineret visningsudfører

Samme visningsudfører fra betinget arbejdsgang - giver endelige resultater som arbejdsgangsoutput.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Trin 7: Indlæs miljøvariabler

Konfigurer LLM-klienten (GitHub Models, Azure OpenAI eller OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Trin 8: Opret AI-agenter og eksekutorer

Vi opretter **seks workflow-komponenter**:

**Agenter (pakket ind i AgentExecutor):**
1. **availability_agent** - Tjekker hoteltilgængelighed ved hjælp af værktøjet
2. **confirmation_agent** - 🆕 Forbereder anmodningen om menneskelig bekræftelse
3. **alternative_agent** - Foreslår alternative byer (når brugeren siger ja)
4. **booking_agent** - Opmuntrer til booking (når værelser er tilgængelige)
5. **cancellation_agent** - 🆕 Håndterer annulleringsbeskeden (når brugeren siger nej)

**Specielle eksekutorer:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, der pauser workflowet for menneskelig input
7. **decision_manager** - 🆕 Brugerdefineret eksekutor, der dirigerer baseret på menneskelig respons (allerede defineret ovenfor)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Trin 9: Byg arbejdsgangen med menneske-i-løkken

Nu konstruerer vi arbejdsgangens graf med **betinget routing**, inklusive menneske-i-løkken-stien:

**Arbejdsgangsstruktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Vigtige forbindelser:**
- `availability_agent → confirmation_agent` (når der ikke er værelser)
- `confirmation_agent → prepare_human_request` (transformer type)
- `prepare_human_request → request_info_executor` (pause for menneske)
- `request_info_executor → decision_manager` (altid - leverer RequestResponse)
- `decision_manager → alternative_agent` (når brugeren siger "ja")
- `decision_manager → cancellation_agent` (når brugeren siger "nej")
- `availability_agent → booking_agent` (når der er værelser tilgængelige)
- Alle stier ender ved `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Trin 10: Kør Test Case 1 - By UDEN Tilgængelighed (Paris med Menneskelig Bekræftelse)

Denne test demonstrerer den **fulde menneske-i-løkken cyklus**:

1. Forespørg hotel i Paris
2. availability_agent tjekker → Ingen værelser
3. confirmation_agent opretter spørgsmål rettet mod mennesket
4. request_info_executor **pauser arbejdsgangen** og udsender `RequestInfoEvent`
5. **Applikationen registrerer hændelsen og beder brugeren om input i konsollen**
6. Brugeren skriver "ja" eller "nej"
7. Applikationen sender svar via `send_responses_streaming()`
8. decision_manager dirigerer baseret på svaret
9. Endeligt resultat vises

**Nøglemønster:**
- Brug `workflow.run_stream()` til første iteration
- Brug `workflow.send_responses_streaming(pending_responses)` til efterfølgende iterationer
- Lyt efter `RequestInfoEvent` for at registrere, hvornår menneskelig input er nødvendig
- Lyt efter `WorkflowOutputEvent` for at fange endelige resultater


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Trin 11: Kør Test Case 2 - By MED Tilgængelighed (Stockholm - Ingen Menneskelig Indgriben Nødvendig)

Denne test demonstrerer den **direkte vej**, når værelser er tilgængelige:

1. Forespørg hotel i Stockholm
2. availability_agent tjekker → Værelser tilgængelige ✅
3. booking_agent foreslår booking
4. display_result viser bekræftelse
5. **Ingen menneskelig indgriben nødvendig!**

Arbejdsgangen springer helt over den menneske-i-løkken vej, når værelser er tilgængelige.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Vigtige Punkter og Bedste Praksis for Human-in-the-Loop

### ✅ Hvad Du Har Lært:

#### 1. **RequestInfoExecutor Mønster**
Human-in-the-loop mønsteret i Microsoft Agent Framework bruger tre nøglekomponenter:
- `RequestInfoExecutor` - Pauser arbejdsgangen og udsender begivenheder
- `RequestInfoMessage` - Basisklasse for typede anmodningsdata (subklassér denne!)
- `RequestResponse` - Knytter menneskelige svar til de oprindelige anmodninger

**Vigtig Forståelse:**
- `RequestInfoExecutor` indsamler IKKE input selv - den pauser kun arbejdsgangen
- Din applikationskode skal lytte efter `RequestInfoEvent` og indsamle input
- Du skal kalde `send_responses_streaming()` med en dict, der kortlægger `request_id` til brugerens svar

#### 2. **Streaming Execution Mønster**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Event-Drevet Arkitektur**
Lyt efter specifikke begivenheder for at styre arbejdsgangen:
- `RequestInfoEvent` - Menneskeligt input er nødvendigt (arbejdsgang pauset)
- `WorkflowOutputEvent` - Endeligt resultat er tilgængeligt (arbejdsgang fuldført)
- `WorkflowStatusEvent` - Statusændringer (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, osv.)

#### 4. **Tilpassede Executors med @handler**
`DecisionManager` demonstrerer, hvordan man opretter executors, der:
- Bruger `@handler` dekorator til at eksponere metoder som arbejdsgangstrin
- Modtager typede beskeder (f.eks. `RequestResponse[HumanFeedbackRequest, str]`)
- Styrer arbejdsgangen ved at sende beskeder til andre executors
- Får adgang til kontekst via `WorkflowContext`

#### 5. **Betinget Routing med Menneskelige Beslutninger**
Du kan oprette betingelsesfunktioner, der evaluerer menneskelige svar:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Virkelige Anvendelser:

1. **Godkendelsesarbejdsgange**
   - Få lederens godkendelse før behandling af udgiftsrapporter
   - Kræv menneskelig gennemgang før afsendelse af automatiserede e-mails
   - Bekræft transaktioner med høj værdi før udførelse

2. **Indholdsmoderering**
   - Marker tvivlsomt indhold til menneskelig gennemgang
   - Bed moderatorer om at træffe endelige beslutninger i grænsetilfælde
   - Eskaler til mennesker, når AI's selvtillid er lav

3. **Kundeservice**
   - Lad AI håndtere rutinemæssige spørgsmål automatisk
   - Eskaler komplekse problemer til menneskelige agenter
   - Spørg kunden, om de ønsker at tale med et menneske

4. **Databehandling**
   - Bed mennesker om at løse tvetydige dataindtastninger
   - Bekræft AI's fortolkninger af uklare dokumenter
   - Lad brugere vælge mellem flere gyldige fortolkninger

5. **Sikkerhedskritiske Systemer**
   - Kræv menneskelig bekræftelse før irreversible handlinger
   - Få godkendelse før adgang til følsomme data
   - Bekræft beslutninger i regulerede industrier (sundhedsvæsen, finans)

6. **Interaktive Agenter**
   - Byg samtalebots, der stiller opfølgende spørgsmål
   - Skab guider, der leder brugere gennem komplekse processer
   - Design agenter, der samarbejder med mennesker trin for trin

### 🔄 Sammenligning: Med vs Uden Human-in-the-Loop

| Funktion | Betinget Arbejdsgang | Human-in-the-Loop Arbejdsgang |
|----------|-----------------------|-------------------------------|
| **Udførelse** | Enkelt `workflow.run()` | Loop med `run_stream()` + `send_responses_streaming()` |
| **Brugerinput** | Ingen (fuldt automatiseret) | Interaktive prompts via `input()` eller UI |
| **Komponenter** | Agenter + Executors | + RequestInfoExecutor + DecisionManager |
| **Begivenheder** | Kun AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, osv. |
| **Pauser** | Ingen pauser | Arbejdsgang pauser ved RequestInfoExecutor |
| **Menneskelig kontrol** | Ingen menneskelig kontrol | Mennesker træffer vigtige beslutninger |
| **Anvendelse** | Automatisering | Samarbejde & overvågning |

### 🚀 Avancerede Mønstre:

#### Flere Menneskelige Beslutningspunkter
Du kan have flere `RequestInfoExecutor` noder i samme arbejdsgang:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Timeout Håndtering
Implementér tidsgrænser for menneskelige svar:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integration med Avanceret UI
I stedet for `input()`, integrer med web UI, Slack, Teams, osv.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Betinget Human-in-the-Loop
Spørg kun om menneskeligt input i specifikke situationer:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Bedste Praksis:

1. **Subclass Altid RequestInfoMessage**
   - Giver type-sikkerhed og validering
   - Muliggør rig kontekst for UI-rendering
   - Klargør formålet med hver anmodningstype

2. **Brug Beskrivende Prompter**
   - Inkluder kontekst om, hvad du spørger om
   - Forklar konsekvenserne af hvert valg
   - Hold spørgsmål enkle og klare

3. **Håndter Uventet Input**
   - Validér brugerens svar
   - Giv standarder for ugyldigt input
   - Giv klare fejlmeddelelser

4. **Spor Anmodnings-ID'er**
   - Brug korrelationen mellem request_id og svar
   - Forsøg ikke at administrere tilstand manuelt

5. **Design til Ikke-Blokering**
   - Bloker ikke tråde, mens du venter på input
   - Brug asynkrone mønstre overalt
   - Understøt samtidige arbejdsgangsinstanser

### 📚 Relaterede Koncepter:

- **Agent Middleware** - Opfang agentkald (tidligere notebook)
- **Arbejdsgangstilstandshåndtering** - Bevar arbejdsgangstilstand mellem kørsel
- **Multi-Agent Samarbejde** - Kombinér human-in-the-loop med agentteams
- **Event-Drevne Arkitekturer** - Byg reaktive systemer med begivenheder

---

### 🎓 Tillykke!

Du har mestret human-in-the-loop arbejdsgange med Microsoft Agent Framework! Du ved nu, hvordan man:
- ✅ Pauser arbejdsgange for at indsamle menneskeligt input
- ✅ Bruger RequestInfoExecutor og RequestInfoMessage
- ✅ Håndterer streaming-udførelse med begivenheder
- ✅ Opretter tilpassede executors med @handler
- ✅ Styrer arbejdsgange baseret på menneskelige beslutninger
- ✅ Bygger interaktive AI-agenter, der samarbejder med mennesker

**Dette er et kritisk mønster for at bygge pålidelige, kontrollerbare AI-systemer!** 🚀



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
